In [1]:
!pip install pymorphy3

import tensorflow as tf
import pymorphy3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Conv1D, GlobalMaxPooling1D, TimeDistributed, Flatten, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from sklearn.utils.class_weight import compute_class_weight
import nltk
from nltk import word_tokenize
import numpy as np
import os
import re
nltk.download('punkt')
nltk.download('punkt_tab')

2025-11-26 13:06:09.151425: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-26 13:06:09.504103: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-26 13:06:10.965299: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
[nltk_data] Downloading package punkt to /home/dream_on/nltk_data...
[nltk_data]   Package 

True

In [2]:
morph = pymorphy3.MorphAnalyzer()
max_len = 0

In [3]:
int_to_str = ['NOUN',
            'ADJF',
            'ADJS',
            'VERB',
            'INFN',
            'NUMR',
            'PRTF',
            'PRTS',
            'GRND',
            'PREP',
            'PRCL',
            'CONJ',
            'NPRO',
            'ADVB',
            'COMP',
            'NONE',
            'PRED',
            'INTJ']
str_to_int = {}
for i, j in enumerate(int_to_str):
  str_to_int[j] = i

character_count = 35 # количество символов - 33 буквы, ' ' и '-'

In [4]:
def process_data(sentence):
  sentence += ' '
  global max_len
  word = ""
  x = []
  y = []
  cnt = 0
  for ch in sentence:
    ch = ch.lower()
    if ord(ch) >= ord('а') and ord(ch) <= ord('я'):
      word += ch
      ohe = [0 for _ in range(character_count)]
      ohe[ord(ch) - ord('а')] = 1
      x.append(np.array(ohe))
      y.append('NONE')
    elif ch == '-':
      word += ch
      ohe = [0 for _ in range(character_count)]
      ohe[character_count - 2] = 1
      x.append(np.array(ohe))
      y.append('NONE')
    elif ch == ' ': # дошли до конца слова
      if word == '':
        continue
      ohe = [0 for _ in range(character_count)]
      ohe[character_count - 1] = 1
      x.append(np.array(ohe))
      y.append(morph.parse(word)[0].tag.POS) # делаем морфлогический разбор, достаем тег части речи
      word = ""
  y = ['NONE' if tag is None else tag for tag in y]
  y = [str_to_int[tag] for tag in y]
  max_len = max(max_len, len(x))
  # print(y)
  return x, y

In [7]:
X = []
Y = []
books = ["MasterAndMargarita.txt"]

for book in books:
    with open("./files/books_before/" + book,"r") as f:
        content = f.read()
    raw_sentences = re.split(r'[.!?]', content) # разбиваем на предложения
    for sentence in raw_sentences:
      x, y = process_data(sentence)
      X.append(x)
      Y.append(y)
    print(f"{book} is done reading")
    
for i in range(len(X)):
  while len(X[i]) < max_len:
    X[i].append(np.array([0 for _ in range(character_count)])) # добиваем ohe предложения до максимальной длины слова нулями (т.е. никакой из нужных символов)
    Y[i].append(str_to_int['NONE']) # И соответственно тег части речи отсутствует
  X[i] = np.array(X[i])
  Y[i] = np.array(Y[i])

MasterAndMargarita.txt is done reading


In [8]:
max_len

567

In [11]:
num_classes = len(str_to_int)

Y_arr = np.array(Y, dtype='int32')      # (N, max_len)
y_flat = Y_arr.flatten()              # (N * max_len,)
classes = np.arange(num_classes)

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=classes,
    y=y_flat
)

class_weight_dict = {i: w for i, w in enumerate(class_weights)}
pad_idx = str_to_int['NONE']
class_weight_dict[pad_idx] = 0.1
sample_weight = np.zeros_like(Y_arr, dtype='float32')  # (N, max_len)

for cls_idx, w in class_weight_dict.items():
    sample_weight[Y_arr == cls_idx] = w


In [13]:
lstm_model = Sequential()
lstm_model.add(Bidirectional(LSTM(units=64, input_shape=(None, max_len, character_count), return_sequences=True)))
lstm_model.add(TimeDistributed(Dense(len(str_to_int), activation='softmax')))

lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

lstm_model.fit(np.array(X), np.array(Y), epochs=5, batch_size=32, validation_split=0.2, sample_weight=sample_weight)

2025-11-26 11:16:04.332546: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/dream_on/miniconda3/lib/python3.13/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
268/268 ━━━━━━━━━━━━━━━━━━━━ 145s 498ms/step - accuracy: 0.8672 - loss: 2.4248 - val_accuracy: 0.8562 - val_loss: 2.3636
Epoch 2/5
268/268 ━━━━━━━━━━━━━━━━━━━━ 130s 484ms/step - accuracy: 0.8825 - loss: 1.5552 - val_accuracy: 0.8751 - val_loss: 1.5166
Epoch 3/5
268/268 ━━━━━━━━━━━━━━━━━━━━ 129s 481ms/step - accuracy: 0.9158 - loss: 1.1146 - val_accuracy: 0.9245 - val_loss: 1.2189
Epoch 4/5
268/268 ━━━━━━━━━━━━━━━━━━━━ 130s 483ms/step - accuracy: 0.9558 - loss: 0.9188 - val_accuracy: 0.9600 - val_loss: 1.0671
Epoch 5/5
268/268 ━━━━━━━━━━━━━━━━━━━━ 129s 482ms/step - accuracy: 0.9743 - loss: 0.7812 - val_accuracy: 0.9784 - val_loss: 0.9318


In [14]:
lstm_model.save('model_lstm.keras')

In [4]:
lstm_model = load_model('model_lstm.keras')

2025-11-26 12:52:47.477055: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [7]:
s = 'Он сбросил с себя шинель и так весело, таким молоденьким мальчиком посмотрел на отца '
s = re.sub(r'[^\w\s]', '', s)
x, y = process_data(s)
while len(x) < max_len:
  x.append(np.zeros(character_count))
  y.append(str_to_int['NONE'])
print(np.array(x).shape)
ans = lstm_model.predict(np.array([x]))
for i in range(len(s)):
  tmp = int_to_str[np.argmax(ans[0][i])]
  print(s[i], end = '')
  if s[i] == ' ':
    print('-', tmp)

(84, 35)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 488ms/step
Он - NPRO
сбросил - VERB
с - PREP
себя - NPRO
шинель - VERB
и - CONJ
так - CONJ
весело - VERB
таким - ADJF
молоденьким - ADJF
мальчиком - ADJF
посмотрел - VERB
на - PREP
отца - NOUN


In [5]:
max_len_word = 26

def process_data_words(sentence):
  sentence += ' '
  global max_len 
  word = ""
  x = []
  y = []
  cnt = 0
  word_tmp = []
  for ch in sentence:
    ch = ch.lower()
    if ord(ch) >= ord('а') and ord(ch) <= ord('я'):
      word += ch
      ohe = [0 for _ in range(character_count)]
      ohe[ord(ch) - ord('а')] = 1
      word_tmp.append(np.array(ohe))
    elif ch == '-':
      word += ch
      ohe = [0 for _ in range(character_count)]
      ohe[character_count - 2] = 1
      word_tmp.append(np.array(ohe))
    elif ch == ' ': # дошли до конца слова
      if word == '':
        continue
      ohe = [0 for _ in range(character_count)]
      ohe[character_count - 1] = 1
      while len(word_tmp) < max_len_word:
        word_tmp.append(np.array(ohe))
      x.append(word_tmp)
      y.append(morph.parse(word)[0].tag.POS) # делаем морфлогический разбор, достаем тег части речи
      word = ""
      word_tmp = []
  y = ['NONE' if tag is None else tag for tag in y]
  y = [str_to_int[tag] for tag in y]
  max_len = max(max_len, len(x))
  return x, y

In [6]:
max_len = 0
X = []
Y = []
books = ["MasterAndMargarita.txt"]

for book in books:
    with open("./files/books_before/" + book,"r") as f:
        content = f.read()
    raw_sentences = re.split(r'[.!?]', content) # разбиваем на предложения
    for sentence in raw_sentences:
      x, y = process_data_words(sentence)
      X.append(x)
      Y.append(y)
    print(f"{book} is done reading")

for i in range(len(X)):
  while len(X[i]) < max_len:
    X[i].append([np.array([0 for _ in range(character_count)]) for _ in range(max_len_word)])
    Y[i].append(str_to_int['NONE'])
  X[i] = np.array(X[i])
  Y[i] = np.array(Y[i])
print()

MasterAndMargarita.txt is done reading



In [10]:
num_classes = len(str_to_int)

Y_arr = np.array(Y, dtype='int32')
y_flat = Y_arr.reshape(-1)
classes = np.arange(num_classes)

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=classes,
    y=y_flat
)

class_weight_dict = {i: w for i, w in enumerate(class_weights)}
pad_idx = str_to_int['NONE']
class_weight_dict[pad_idx] = 0.1
sample_weight = np.zeros_like(Y_arr, dtype='float32')

for cls_idx, w in class_weight_dict.items():
    sample_weight[Y_arr == cls_idx] = w


In [8]:
import tensorflow as tf
from tensorflow.keras import layers, Model, Input

inputs = Input(shape=(max_len, max_len_word, character_count))

# 1. Свёртка по символам каждого слова
char_conv = layers.TimeDistributed(
    layers.Conv1D(filters=64,
                  kernel_size=3,
                  padding='same',
                  activation='relu')
)(inputs)

# 2. Пулинг по символам → вектор на слово
word_vectors = layers.TimeDistributed(layers.GlobalMaxPooling1D())(char_conv)

# 3. Свёртка по словам
context = layers.Conv1D(filters=128, kernel_size=3, padding='same', activation='relu')(word_vectors)

# 4. Dense для каждого слова
dense_out = layers.TimeDistributed(
    layers.Dense(64, activation='relu')
)(context)

# 5. Выход: POS-теги для каждого слова
outputs = layers.TimeDistributed(
    layers.Dense(len(str_to_int), activation='softmax')
)(dense_out)

model = Model(inputs, outputs)
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


2025-11-26 13:08:39.561015: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 82, 26, 35)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 82, 26, 64)     │         6,784 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 82, 64)         │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 82, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 82, 64)         │         8,256 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_3              │ (None, 82, 18)         │         1,170 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 40,914 (159.82 KB)

 Trainable params: 40,914 (159.82 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
model.fit(np.array(X), np.array(Y), epochs=5, batch_size=32, validation_split=0.2, sample_weight=sample_weight)

Epoch 1/5
268/268 ━━━━━━━━━━━━━━━━━━━━ 45s 141ms/step - accuracy: 0.8953 - loss: 1.9036 - val_accuracy: 0.9380 - val_loss: 1.2857
Epoch 2/5
268/268 ━━━━━━━━━━━━━━━━━━━━ 35s 132ms/step - accuracy: 0.9513 - loss: 0.8445 - val_accuracy: 0.9513 - val_loss: 0.8968
Epoch 3/5
268/268 ━━━━━━━━━━━━━━━━━━━━ 36s 134ms/step - accuracy: 0.9648 - loss: 0.5786 - val_accuracy: 0.9627 - val_loss: 0.6427
Epoch 4/5
268/268 ━━━━━━━━━━━━━━━━━━━━ 37s 139ms/step - accuracy: 0.9721 - loss: 0.4396 - val_accuracy: 0.9688 - val_loss: 0.5355
Epoch 5/5
268/268 ━━━━━━━━━━━━━━━━━━━━ 37s 136ms/step - accuracy: 0.9762 - loss: 0.3510 - val_accuracy: 0.9729 - val_loss: 0.4741


In [12]:
model.save('model_cnn.keras')

In [13]:
cnn_model = load_model('model_cnn.keras')

In [14]:
s = 'Он сбросил с себя шинель и так весело, таким молоденьким мальчиком посмотрел на отца '
s = re.sub(r'[^\w\s]', '', s)
x, y = process_data_words(s)
while len(x) < max_len:
    x.append([np.array([0 for _ in range(character_count)]) for _ in range(max_len_word)])
    y.append(str_to_int['NONE'])
ans = cnn_model.predict(np.array([x]))

j=0
for i in range(len(s)):
  tmp = int_to_str[np.argmax(ans[0][j])]
  print(s[i], end = '')
  if s[i] == ' ':
    j+=1
    print('-', tmp)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Он - NPRO
сбросил - VERB
с - PREP
себя - NPRO
шинель - NOUN
и - CONJ
так - CONJ
весело - ADJS
таким - ADJF
молоденьким - ADJF
мальчиком - NOUN
посмотрел - VERB
на - PREP
отца - NOUN
